If you are using this notebook on google colab, you might want to uncomment the following two cells

In [1]:
# from google.colab import userdata

In [2]:
# !pip install "dspy>=3.0.3" sacrebleu tqdm rich

## LM Setup and Intro

The API key for [ai.ufal.mff.cuni.cz](https://chat.ai.e-infra.cz/) can be found like this:
1. Click on the profile picture at the top right corner
2. Select Settings
3. Select the Account tab
4. Expand API keys
5. Click the Copy to Clipboard icon

You should copy the key and add it as a new Google Colab secret (left row, icon with a key) under the name "CHAT_AI_API_KEY". If using the notebook locally, you can store the keys in files, for example in ~/.ssh/chat_ai_api.key, ideally, this key should be set to be readable only by you current user. You could also directly copy the key and use it as a string here, but it is a bad practice in case you forget about it and share the Jupyter notebook with someone later (e.g. through git).

In [3]:
import dspy

# OpenAI servers
# lm = dspy.LM("gpt-4o-mini", api_key=userdata.get("OPENAI_API_KEY"))

# CESNET servers (https://docs.cerit.io/en/docs/web-apps/chat-ai)
lm = dspy.LM(
    "openai/gpt-oss-120b",
    base_url="https://llm.ai.e-infra.cz/v1/",
    # api_key=userdata.get("CHAT_AI_API_KEY")  # load the API key from Colab's user data
    api_key=open("/home/mhn/.ssh/chat_ai_api.key").read().strip(),  # load the API key from a local file
    max_length=16000, # we increase the token limit to 16k, because gpt120-oss takes up a lot of tokens when thinking
)

dspy.configure(lm=lm)

In [4]:
lm("Hello world!")

[{'text': 'Hello! 👋 How can I assist you today?',
  'reasoning_content': 'The user just says "Hello world!" Probably a greeting. We should respond friendly. Maybe ask how we can help.'}]

DSPy uses cache by default. If you run the exact same request multiple times, you will get the same answer.

In [5]:
print(lm("Why is the sky blue!", temperature=0.7)[0])

{'text': '**Short answer:**  \nThe sky looks blue because the Earth’s atmosphere scatters short‑wavelength (blue‑violet) light from the Sun much more efficiently than it scatters longer‑wavelength (red, orange, yellow) light. Our eyes perceive that scattered blue light coming from every direction overhead.\n\n---\n\n## The physics behind the color\n\n| Step | What happens | Why it matters |\n|------|--------------|----------------|\n| **1. Sunlight is white** | Sunlight contains a continuous spectrum of colors (red\u202f→\u202fviolet). | “White” light is just a mix of all visible wavelengths. |\n| **2. Light enters the atmosphere** | Photons encounter molecules (N₂, O₂) and tiny particles. | These scatter the light in all directions. |\n| **3. Rayleigh scattering** | For particles much smaller than the light’s wavelength, the scattering intensity follows an inverse‑fourth‑power law:\u202f\\(I \\propto 1/λ^{4}\\). | Shorter wavelengths (blue ~450\u202fnm, violet ~400\u202fnm) are scatte

In [6]:
lm("Why is the sky blue!", temperature=0.1)

[{'text': '**Short answer:**  \nThe sky looks blue because the Earth’s atmosphere scatters short‑wavelength (blue‑violet) sunlight much more efficiently than it scatters longer‑wavelength (red, orange, yellow) light. Our eyes end up receiving a lot of that scattered blue light from every direction.\n\n---\n\n### The physics in a nutshell  \n\n1. **Sunlight is a mixture of colors**  \n   Sunlight contains all visible wavelengths (roughly 380\u202fnm\u202f–\u202f750\u202fnm). If you pass it through a prism you see a rainbow, which shows that the Sun emits roughly equal amounts of red, orange, yellow, green, blue, indigo, and violet light.\n\n2. **What the atmosphere does: Rayleigh scattering**  \n   - The air is filled with molecules (N₂, O₂, etc.) that are *much* smaller than the wavelength of visible light.  \n   - When light hits these tiny particles, it is scattered in all directions.  \n   - The scattering intensity follows **Rayleigh’s law**, which says the amount of scattering is 

If you want to sample multiple responses with the same sampling parameters, you can use `rollout_id`. You might get different reponses, but they will still get cached for a given `rollout_id` Note that this only makes sense with sampling (e.g. temperature > 0).

In [7]:
lm("Why is the sky blue!", temperature=0.7, rollout_id=1)

[{'text': '**Short answer:**  \nThe sky looks blue because the Earth’s atmosphere scatters short‑wavelength (blue‑violet) light from the Sun much more efficiently than it scatters longer‑wavelength (red, orange, yellow) light. Our eyes perceive that scattered blue light coming from every direction overhead.\n\n---\n\n## The physics behind the color\n\n| Step | What happens | Why it matters |\n|------|--------------|----------------|\n| **1. Sunlight is white** | Sunlight contains a continuous spectrum of colors (red\u202f→\u202fviolet). | “White” light is just a mix of all visible wavelengths. |\n| **2. Light enters the atmosphere** | Photons encounter molecules (N₂, O₂) and tiny particles. | These scatter the light in all directions. |\n| **3. Rayleigh scattering** | For particles much smaller than the light’s wavelength, the scattering intensity follows an inverse‑fourth‑power law:\u202f\\(I \\propto 1/λ^{4}\\). | Shorter wavelengths (blue ~450\u202fnm, violet ~400\u202fnm) are scatt

In [8]:
lm("Why is the sky blue!", temperature=0.7, rollout_id=2)

[{'text': '**Short answer:**  \nThe sky looks blue because the Earth’s atmosphere scatters short‑wavelength (blue‑violet) light from the Sun much more efficiently than it scatters longer‑wavelength (red, orange, yellow) light. Our eyes perceive that scattered blue light coming from every direction overhead.\n\n---\n\n## The physics behind the color\n\n| Step | What happens | Why it matters |\n|------|--------------|----------------|\n| **1. Sunlight is white** | Sunlight contains a continuous spectrum of colors (red\u202f→\u202fviolet). | “White” light is just a mix of all visible wavelengths. |\n| **2. Light enters the atmosphere** | Photons encounter molecules (N₂, O₂) and tiny particles. | These scatter the light in all directions. |\n| **3. Rayleigh scattering** | For particles much smaller than the light’s wavelength, the scattering intensity follows an inverse‑fourth‑power law:\u202f\\(I \\propto 1/λ^{4}\\). | Shorter wavelengths (blue ~450\u202fnm, violet ~400\u202fnm) are scatt

## DSPy Signatures

Signatures specify the inputs and outputs and their types.

In [9]:
from typing import Literal
# string based definition
Translate1 = "src_lang, tgt_lang, src -> tgt"

# class based definition (can contain initial prompt, description of the fields, etc.)
class Translate2(dspy.Signature):
    """Translate the src text from src_lang to tgt_lang."""
    src_lang: str = dspy.InputField(desc="Source language")
    tgt_lang: str = dspy.InputField(desc="Target language")
    src: str = dspy.InputField()

    tgt: str = dspy.OutputField()
    terminology_list: list[tuple[str, str, Literal["standard translation", "transliteration", "other"]]] = dspy.OutputField(desc="list of triplets of (src term, tgt term, reasoning)")

translate2 = dspy.Predict(Translate2)
pred = translate2(src_lang="English", tgt_lang="French", src="""Prague (/ˈprɑːɡ/ PRAHG; Czech: Praha [ˈpraɦa] ⓘ)[a] is the capital and largest city of the Czech Republic[9] and the historical capital of Bohemia. Prague, located on the Vltava River, has a population of about 1.4 million, while its metropolitan area is home to approximately 2.3 million people.""")

In [10]:
print(pred)

Prediction(
    tgt='Prague (/ˈprɑːɡ/ PRAHG\u202f; tchèque\u202f: Praha [ˈpraɦa] ⓘ)[a] est la capitale et la plus grande ville de la République tchèque et l’ancienne capitale historique de la Bohême. Prague, située sur le fleuve Vltava, compte une population d’environ 1,4\u202fmillion d’habitants, tandis que son agglomération abrite environ\u202f2,3\u202fmillions de personnes.',
    terminology_list=[('Prague', 'Prague', 'standard translation'), ('Czech Republic', 'République tchèque', 'standard translation'), ('Bohemia', 'Bohême', 'standard translation'), ('Vltava River', 'rivière Vltava', 'standard translation')]
)


In [11]:
dspy.inspect_history()





[2026-02-27T02:48:30.892970]

System message:

Your input fields are:
1. `src_lang` (str): Source language
2. `tgt_lang` (str): Target language
3. `src` (str):
Your output fields are:
1. `tgt` (str): 
2. `terminology_list` (list[tuple[str, str, Literal['standard translation', 'transliteration', 'other']]]): list of triplets of (src term, tgt term, reasoning)
All interactions will be structured in the following way, with the appropriate values filled in.

Inputs will have the following structure:

[[ ## src_lang ## ]]
{src_lang}

[[ ## tgt_lang ## ]]
{tgt_lang}

[[ ## src ## ]]
{src}

Outputs will be a JSON object with the following fields.

{
  "tgt": "{tgt}",
  "terminology_list": "{terminology_list}        # note: the value you produce must adhere to the JSON schema: {\"type\": \"array\", \"items\": {\"type\": \"array\", \"maxItems\": 3, \"minItems\": 3, \"prefixItems\": [{\"type\": \"string\"}, {\"type\": \"string\"}, {\"type\": \"string\", \"enum\": [\"standard translation\", \

In [12]:
# To use signatures, we can wrap them in built-in modules, such as dspy.Predict and dspy.ChainOfThought
translate1 = dspy.Predict(Translate1)
translate1_cot = dspy.ChainOfThought(Translate1)
translate2 = dspy.Predict(Translate2)

In [13]:
translate1(src_lang="English", tgt_lang="Czech", src="Prague is the capital of the Czech Republic.")

Prediction(
    tgt='Praha je hlavní město České republiky.'
)

In [14]:
translate1_cot(src_lang="English", tgt_lang="Czech", src="Prague is the capital of the Czech Republic.")

Prediction(
    reasoning='The sentence states that Prague is the capital of the Czech Republic. In Czech, "Prague" is "Praha", "is" translates to the verb "je", "the capital" becomes "hlavním městem", and "of the Czech Republic" is "České republiky". The proper case and word order for this statement in Czech is: "Praha je hlavním městem České republiky."',
    tgt='Praha je hlavním městem České republiky.'
)

Let's explore what does the actual prompt sent to the LLM look like.

In [15]:
import json
adapter = dspy.ChatAdapter()
print(json.dumps(adapter.format(translate2.signature, demos=[], inputs=dict(src_lang="English", tgt_lang="Czech", src="Prague is the capital of the Czech Republic.")), indent=2))

[
  {
    "role": "system",
    "content": "Your input fields are:\n1. `src_lang` (str): Source language\n2. `tgt_lang` (str): Target language\n3. `src` (str):\nYour output fields are:\n1. `tgt` (str): \n2. `terminology_list` (list[tuple[str, str, Literal['standard translation', 'transliteration', 'other']]]): list of triplets of (src term, tgt term, reasoning)\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## src_lang ## ]]\n{src_lang}\n\n[[ ## tgt_lang ## ]]\n{tgt_lang}\n\n[[ ## src ## ]]\n{src}\n\n[[ ## tgt ## ]]\n{tgt}\n\n[[ ## terminology_list ## ]]\n{terminology_list}        # note: the value you produce must adhere to the JSON schema: {\"type\": \"array\", \"items\": {\"type\": \"array\", \"maxItems\": 3, \"minItems\": 3, \"prefixItems\": [{\"type\": \"string\"}, {\"type\": \"string\"}, {\"type\": \"string\", \"enum\": [\"standard translation\", \"transliteration\", \"other\"]}]}}\n\n[[ ## completed ## ]]\nIn adhering to t

In [16]:
for message in adapter.format(translate1.signature, demos=[], inputs=dict(src_lang="English", tgt_lang="Czech", src="Prague is the capital of the Czech Republic.")):
  print(f"**{message['role']}**")
  print(message['content'])
  print("\n")

**system**
Your input fields are:
1. `src_lang` (str): 
2. `tgt_lang` (str): 
3. `src` (str):
Your output fields are:
1. `tgt` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## src_lang ## ]]
{src_lang}

[[ ## tgt_lang ## ]]
{tgt_lang}

[[ ## src ## ]]
{src}

[[ ## tgt ## ]]
{tgt}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `src_lang`, `tgt_lang`, `src`, produce the fields `tgt`.


**user**
[[ ## src_lang ## ]]
English

[[ ## tgt_lang ## ]]
Czech

[[ ## src ## ]]
Prague is the capital of the Czech Republic.

Respond with the corresponding output fields, starting with the field `[[ ## tgt ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.




Let's also look at the variant with the initial prompt

In [17]:
for message in adapter.format(translate2.signature, demos=[], inputs=dict(src_lang="English", tgt_lang="Czech", src="Prague is the capital of the Czech Republic.")):
  print(f"**{message['role']}**")
  print(message['content'])
  print("\n")

**system**
Your input fields are:
1. `src_lang` (str): Source language
2. `tgt_lang` (str): Target language
3. `src` (str):
Your output fields are:
1. `tgt` (str): 
2. `terminology_list` (list[tuple[str, str, Literal['standard translation', 'transliteration', 'other']]]): list of triplets of (src term, tgt term, reasoning)
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## src_lang ## ]]
{src_lang}

[[ ## tgt_lang ## ]]
{tgt_lang}

[[ ## src ## ]]
{src}

[[ ## tgt ## ]]
{tgt}

[[ ## terminology_list ## ]]
{terminology_list}        # note: the value you produce must adhere to the JSON schema: {"type": "array", "items": {"type": "array", "maxItems": 3, "minItems": 3, "prefixItems": [{"type": "string"}, {"type": "string"}, {"type": "string", "enum": ["standard translation", "transliteration", "other"]}]}}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Translate the src text from src_lang to tgt_lang.



## Metrics

Before we get around to using optimizers, we have to specify some metrics we can optimize. Let's try using our LM to also score the translations.

In [18]:
class EvaluateTranslation(dspy.Signature):
    """Assign an integer score (0-10) to the translation. 0 - completely wrong, 10 - perfect."""
    src_lang: str = dspy.InputField()
    tgt_lang: str = dspy.InputField()
    src: str = dspy.InputField()
    tgt: str = dspy.InputField()

    score: int = dspy.OutputField()
evaluate_translation = dspy.Predict(EvaluateTranslation)

def metric(gold, pred, trace=None):  # -> bool, int, float, higher is better
    src_lang = gold.src_lang
    tgt_lang = gold.tgt_lang
    src = gold.src
    tgt = pred.tgt

    eval_output = evaluate_translation(src_lang=src_lang, tgt_lang=tgt_lang, src=src, tgt=tgt)
    score = eval_output.score

    return score/10

evaluate_translation(src_lang="English", tgt_lang="Czech", src="Prague is the capital of the Czech Republic.", tgt="Praha je hlavním městem České republiky.")

Prediction(
    score=10
)

In [19]:
evaluate_translation(src_lang="English", tgt_lang="Czech", src="Prague is the capital of the Czech Republic.", tgt="Praha je hlavním České republiky.")

Prediction(
    score=2
)

In [20]:
evaluate_translation(src_lang="English", tgt_lang="Czech", src="Prague is the capital of the Czech Republic.", tgt="Praha je hlavním městem České Republiky.")

Prediction(
    score=9
)

## Data preparation

In [21]:
evaluate_translation(src_lang="English", tgt_lang="Czech", src="Prague is the capital of the Czech Republic.", tgt="Praha je městem České republiky.")

Prediction(
    score=3
)

In [22]:
import pandas as pd
# df = pd.read_json("/content/drive/MyDrive/DGT/wmt24_esa.jsonl.zst", lines=True)  # load from google drive
df = pd.read_json("../data/wmt24_esa.jsonl.zst", lines=True)  # load from local file
import random
random.seed(42)
df=df[df["langs"]=="en-cs"]
df=df[df["system"] == "refA"]
data = df.to_dict("records")
random.shuffle(data)

trainset = data[:300]
testset = data[300:]

print(f"{len(trainset)=}, {len(testset)=}")
print(trainset[0])

len(trainset)=300, len(testset)=619
{'langs': 'en-cs', 'line_id': 787, 'src': "Hi everyone, welcome back to its Dwight Cooking Show. Today I'll be giving you a tuna macaroni salad recipe. Nice and easy. Just a simple recipe. Here is a list of my ingredients, my bell pepper, my green onions, white onions, half a teaspoon of salt for taste, black pepper, or if you have ground pepper, you can use it, my macaroni, three eggs that were going to be boiled and my tuna.", 'tgt': 'Ahoj všichni, vítám vás u Dwightova pořadu o vaření. Dnes vás naučím recept na těstovinový salát s tuňákem. Dobrý a jednoduchý. Prostě snadný recept. Tady je seznam přísad: paprika, jarní cibulka, bílá cibule, půl lžičky soli pro lepší chuť, černý pepř, nebo pokud máte mletý pepř, můžete použít ten. Dále těstoviny, tři vejce, která uvaříme, a tuňáka.', 'doc_id': 'test-en-speech_WLS2EoW96t4_000', 'domain': 'speech', 'esa_spans': [], 'esa_score': 100, 'system': 'refA', 'annotator': 'engces7713', 'speech_info': {'file': 

In [23]:
def convert_example(row):
    return dspy.Example(
        src_lang="English",
        tgt_lang="Czech",
        src=row["src"],
        tgt=row["tgt"],
    ).with_inputs("src_lang", "tgt_lang", "src")

trainset = [convert_example(row) for row in trainset]
testset = [convert_example(row) for row in testset]

print(trainset[0])

Example({'src_lang': 'English', 'tgt_lang': 'Czech', 'src': "Hi everyone, welcome back to its Dwight Cooking Show. Today I'll be giving you a tuna macaroni salad recipe. Nice and easy. Just a simple recipe. Here is a list of my ingredients, my bell pepper, my green onions, white onions, half a teaspoon of salt for taste, black pepper, or if you have ground pepper, you can use it, my macaroni, three eggs that were going to be boiled and my tuna.", 'tgt': 'Ahoj všichni, vítám vás u Dwightova pořadu o vaření. Dnes vás naučím recept na těstovinový salát s tuňákem. Dobrý a jednoduchý. Prostě snadný recept. Tady je seznam přísad: paprika, jarní cibulka, bílá cibule, půl lžičky soli pro lepší chuť, černý pepř, nebo pokud máte mletý pepř, můžete použít ten. Dále těstoviny, tři vejce, která uvaříme, a tuňáka.'}) (input_keys={'src', 'tgt_lang', 'src_lang'})


## Optimization

### MIPROv2

In [24]:
optimizer_mipro = dspy.MIPROv2(
    metric=metric,
    auto="light",
    num_threads=16,
)

optimized_program_mipro = optimizer_mipro.compile(
    translate1,
    trainset=trainset,
)

# Save the optimized program for future use
optimized_program_mipro.save("../optimized_b2_mipro.json")

2026/02/27 02:48:33 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 10
minibatch: True
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 100

2026/02/27 02:48:33 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2026/02/27 02:48:33 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2026/02/27 02:48:33 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...


Bootstrapping set 1/6
Bootstrapping set 2/6
Bootstrapping set 3/6


  0%|          | 0/60 [00:00<?, ?it/s]

  7%|▋         | 4/60 [00:00<00:02, 23.17it/s]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Bootstrapping set 4/6


...

2026/02/27 02:54:30 INFO dspy.evaluate.evaluate: Average Metric: 30.7 / 35 (87.7%)
2026/02/27 02:54:30 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 87.71 on minibatch of size 35 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 4'].
2026/02/27 02:54:30 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [83.43, 79.71, 76.57, 82.57, 87.71, 77.71, 83.71, 83.14, 87.71]
2026/02/27 02:54:30 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [84.9, 88.3]
2026/02/27 02:54:30 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 88.3
2026/02/27 02:54:30 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2026/02/27 02:54:30 INFO dspy.teleprompt.mipro_optimizer_v2: == Trial 12 / 13 - Minibatch ==



Average Metric: 30.90 / 35 (88.3%): 100%|██████████| 35/35 [00:00<00:00, 945.87it/s]

2026/02/27 02:54:30 INFO dspy.evaluate.evaluate: Average Metric: 30.9 / 35 (88.3%)
2026/02/27 02:54:30 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 88.29 on minibatch of size 35 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 5'].
2026/02/27 02:54:30 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [83.43, 79.71, 76.57, 82.57, 87.71, 77.71, 83.71, 83.14, 87.71, 88.29]
2026/02/27 02:54:30 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [84.9, 88.3]
2026/02/27 02:54:30 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 88.3
2026/02/27 02:54:30 INFO dspy.teleprompt.mipro_optimizer_v2: ==========================================


2026/02/27 02:54:30 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 13 / 13 - Full Evaluation =====
2026/02/27 02:54:30 INFO dspy.teleprompt.mipro_optimizer_v2: Doing full eval on next top averaging program (Avg Score: 87.71) from minibatch trials...



Average Metric: 86.90 / 100 (86.9%): 100%|██████████| 100/100 [00:48<00:00,  2.07it/s]

2026/02/27 02:55:18 INFO dspy.evaluate.evaluate: Average Metric: 86.9 / 100 (86.9%)
2026/02/27 02:55:18 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [84.9, 88.3, 86.9]
2026/02/27 02:55:18 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 88.3
2026/02/27 02:55:18 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2026/02/27 02:55:18 INFO dspy.teleprompt.mipro_optimizer_v2: 

2026/02/27 02:55:18 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 88.3!


In [25]:
optimized_program_mipro(src_lang="English", tgt_lang="Czech", src="Prague is the capital of the Czech Republic.")

Prediction(
    tgt='Praha je hlavním městem České republiky.'
)

In [26]:
dspy.inspect_history()





[2026-02-27T02:55:19.827694]

System message:

Your input fields are:
1. `src_lang` (str): 
2. `tgt_lang` (str): 
3. `src` (str):
Your output fields are:
1. `tgt` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## src_lang ## ]]
{src_lang}

[[ ## tgt_lang ## ]]
{tgt_lang}

[[ ## src ## ]]
{src}

[[ ## tgt ## ]]
{tgt}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `src_lang`, `tgt_lang`, `src`, produce the fields `tgt`.


User message:

[[ ## src_lang ## ]]
English

[[ ## tgt_lang ## ]]
Czech

[[ ## src ## ]]
Damn this rusty K1100! Is there a single screw which has not seized?!


Assistant message:

[[ ## tgt ## ]]
Sakra, ten rezavý K1100! Existuje vůbec jediný šroub, který se nepřipletl?!

[[ ## completed ## ]]


User message:

[[ ## src_lang ## ]]
English

[[ ## tgt_lang ## ]]
Czech

[[ ## src ## ]]
The thick cloud cover prevented sunlight from reaching the surface of th

### SIMBA

In [ ]:
optimizer_simba = dspy.SIMBA(
    metric=metric,
    num_threads=16,
)

optimized_program_simba = optimizer_simba.compile(
    translate1,
    trainset=trainset,
)

# Save optimize program for future use
optimized_program_simba.save("../optimized_b2_simba.json")

2026/02/27 02:55:19 INFO dspy.teleprompt.simba: Starting batch 1 of 8.
2026/02/27 02:55:20 INFO dspy.teleprompt.simba: Sampling program trajectories on 32 examples x 6 samples.


Processed 43 / 192 examples:  22%|██▏       | 42/192 [00:19<00:20,  7.27it/s]

2026/02/27 02:55:40 WARNING dspy.teleprompt.simba_utils: Adapter JSONAdapter failed to parse the LM response. 

LM Response: {"final":"{ \"score\": 3 }"} 

Expected to find output fields in the LM response: [score] 

Actual output fields parsed from the LM response: [] 




Processed 45 / 192 examples:  23%|██▎       | 44/192 [00:21<00:31,  4.68it/s]

2026/02/27 02:55:42 WARNING dspy.teleprompt.simba_utils: Adapter JSONAdapter failed to parse the LM response. 

LM Response: {"final":{"score":4}} 

Expected to find output fields in the LM response: [score] 

Actual output fields parsed from the LM response: [] 




...

Processed 79 / 192 examples:  41%|████      | 78/192 [00:47<01:37,  1.17it/s]

2026/02/27 03:26:41 WARNING dspy.teleprompt.simba_utils: Adapter JSONAdapter failed to parse the LM response. 

LM Response: {"final": {"score": 9}} 

Expected to find output fields in the LM response: [score] 

Actual output fields parsed from the LM response: [] 




Processed 142 / 192 examples:  74%|███████▍  | 142/192 [01:09<00:18,  2.78it/s]

2026/02/27 03:27:03 WARNING dspy.teleprompt.simba_utils: Adapter JSONAdapter failed to parse the LM response. 

LM Response: {"final": {"score": 9}} 

Expected to find output fields in the LM response: [score] 

Actual output fields parsed from the LM response: [] 




Processed 187 / 192 examples:  97%|█████████▋| 187/192 [01:27<00:03,  1.52it/s]

In [ ]:
optimized_program_simba(src_lang="English", tgt_lang="Czech", src="Prague is the capital of the Czech Republic.")

Prediction(
    tgt='Praha je hlavním městem České republiky.'
)

In [ ]:
dspy.inspect_history()





[2025-09-22T14:38:43.791122]

System message:

Your input fields are:
1. `src_lang` (str): 
2. `tgt_lang` (str): 
3. `src` (str):
Your output fields are:
1. `tgt` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## src_lang ## ]]
{src_lang}

[[ ## tgt_lang ## ]]
{tgt_lang}

[[ ## src ## ]]
{src}

[[ ## tgt ## ]]
{tgt}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `src_lang`, `tgt_lang`, `src`, produce the fields `tgt`.
        
        If the input `src` contains mostly repeated characters and informal language (like 'waaaaahoooo!! lol'), then avoid attempting a translation and instead return the original `src` string as the `tgt`. Focus on preserving the original input when it appears to be non-standard or already in the target language's character set.


User message:

[[ ## src_lang ## ]]
English

[[ ## tgt_lang ## ]]
Czech

[[ ## src ## ]]
Amazing how there's two PKs 

In [ ]:
for message in adapter.format(optimized_program_simba.signature, demos=[], inputs=dict(src_lang="English", tgt_lang="Czech", src="Prague is the capital of the Czech Republic.")):
  print(f"**{message['role']}**")
  print(message['content'])
  print("\n")

## Evaluation

In [ ]:
# this can be used to estimate USD cost for commercial models such as
# gpt-4o-mini, it is not relevant for our local models
cost = sum([x['cost'] for x in lm.history if x['cost'] is not None])
cost

0

In [ ]:
lm.history[-1]

In [ ]:
output_base = translate1.batch(testset, num_threads=16)
output_mipro = optimized_program_mipro.batch(testset, num_threads=16)
output_optimized_simba = optimized_program_simba.batch(testset, num_threads=16)

Processed 619 / 619 examples: 100%|██████████| 619/619 [01:46<00:00,  5.83it/s]


In [ ]:
from sacrebleu import corpus_bleu
print("Base:", corpus_bleu([ex.tgt for ex in testset], [[out.tgt for out in output_base]]))
print("MIPRO:", corpus_bleu([ex.tgt for ex in testset], [[out.tgt for out in output_mipro]]))
print("SIMBA:", corpus_bleu([ex.tgt for ex in testset], [[out.tgt for out in output_optimized_simba]]))

Base: BLEU = 28.72 60.2/34.6/22.2/14.7 (BP = 1.000 ratio = 1.020 hyp_len = 24776 ref_len = 24298)
MIPRO: BLEU = 29.04 60.6/35.0/22.5/14.9 (BP = 1.000 ratio = 1.002 hyp_len = 24776 ref_len = 24734)
SIMBA: BLEU = 28.93 60.2/34.9/22.4/14.9 (BP = 1.000 ratio = 1.026 hyp_len = 24776 ref_len = 24149)


Place {{1}} in the {{2}}.